In [1]:
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import pandas as pd
import numpy as np

plt.style.use(astropy_mpl_style)
%matplotlib inline

In [2]:
image_filename = "../datasets/test.fits"

In [3]:
with fits.open(image_filename) as hdul:
    data = hdul[0].data
    hdr = hdul[0].header

In [4]:
hdr

SIMPLE  =                    T / Written by IDL:  Thu Jan 21 16:46:10 2016      
BITPIX  =                   32 / length of data axis 1                          
NAXIS   =                    2 / data unit checksum updated 2014-10-09T06:38:39 
NAXIS1  =                   20 / IMAGE extension                                
NAXIS2  =                   20 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
DATE    = '2014-10-09T06:38:39' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
TELESCOP= 'MOST    '           / TELESCOPE                                      
DATE-REL= '2035-12-12T00:00:00.00' /Release Date.Default = "12-31-2035T00:00:00.
VRSNARCH= '6.0.0.29'           / ARCHIVE VERSION                                
VRSNFITS=                   

In [ ]:
plt.figure(figsize=(14, 8))
plt.imshow(data, cmap="gray")
plt.colorbar()

plt.title("Observer: {}\nTarget: {}".format(hdr["OBSERVER"], hdr["TARGET"]))
plt.show()

In [ ]:
type(data)

In [ ]:
data.shape

In [ ]:
df = pd.DataFrame(
    np.array(data).byteswap().newbyteorder()
)

In [ ]:
df

In [ ]:
df.values

In [ ]:
df.query()

In [ ]:
hdr

In [ ]:
exposure_date = hdr["DATE-OBS"]
observer = hdr["OBSERVER"]
target = hdr["TARGET"]
amplifier_value = hdr["OPAMP"]
gain = hdr["GAIN"]
magnetic_field = hdr["MAG_FLD"]
error_number = hdr["ERR_NUMB"]
heater_voltage = hdr["HEATER_V"]
ccd_temp = hdr["T_CCD"]
board_temp = hdr["T_BOARD"]
preamp_temp = hdr["T_PREAMP"]

In [ ]:
ccd_temp

In [ ]:
observer

In [ ]:
target

In [ ]:
amplifier_value

In [ ]:
magnetic_field

In [ ]:
values_lst = np.concatenate(data).ravel().tolist()

In [ ]:
values_lst

In [ ]:
threshold_value = 30000

In [ ]:
above_threshold = [
    i for i in values_lst
    if i >= threshold_value
]

In [ ]:
len(above_threshold)

In [ ]:
import os

In [ ]:
import tarfile

In [ ]:
notebook_path = os.path.abspath("exploratory.ipynb")
tar_dir = os.path.join(os.path.dirname(os.path.dirname(notebook_path)), "datasets/")
tar_dir

In [ ]:
import time

In [ ]:
for root, dirs, files in os.walk(tar_dir):
    for f in files:
        if os.path.splitext(f)[1] == ".tar":
            
            with tarfile.open(
                name=os.path.join(root, f),
                mode="r"
            ) as tar_obj:
                
                for member in tar_obj.getnames():
                    extracted = tar_obj.extractfile(member)

                    with fits.open(extracted) as hdul:
                        data = hdul[0].data
                        hdr = hdul[0].header

In [ ]:
plt.figure(figsize=(14, 8))
plt.imshow(data, cmap="gray")
plt.colorbar()

plt.title("Observer: {}\nTarget: {}\nDate: {}".format(hdr["OBSERVER"], hdr["TARGET"], hdr["DATE-OBS"]))
plt.show()

In [ ]:
hdr["ERR_NUMB"]

In [ ]:
from astroquery.alma import Alma

In [ ]:
star = Alma.query_object("HD209548", public=False, science=False)

In [ ]:
star

In [ ]:
from astropy import coordinates, units as u, wcs
from astroquery.skyview import SkyView
from astroquery.vizier import Vizier
from astroquery.mast import Mast, Observations

In [ ]:
results = Observations.query_object("HD209548")

In [ ]:
results

In [ ]:
selected_bands = results[
    (results['obs_collection'] == 'TESS') &
    (results['instrument_name'] == 'Photometer') &
    (results["obsid"] == "17000033636")                    
]
prodlist = Observations.get_product_list(selected_bands)
filtered_prodlist = Observations.filter_products(prodlist)
filtered_prodlist

In [ ]:
def plot_image(image_data):
    plt.figure(figsize=(14, 8))
    plt.imshow(image_data)
    plt.colorbar()

    plt.show()

In [ ]:
center = coordinates.SkyCoord.from_name("HD209548")

# Grab an image from SkyView of the Orion KL nebula region
imglist = SkyView.get_images(position=center, survey=["DSS", "DSS2 IR"])

In [ ]:
imglist

In [ ]:
for img in imglist:
    image_data = img[0].data
    wcs_data = wcs.WCS(img[0].header)
    plot_image(image_data)

In [ ]:
imglist[1]

In [ ]:
img = imglist[0]

# 'img' is now a fits.HDUList object; the 0th entry is the image
mywcs = wcs.WCS(img[0].header)

In [ ]:
mywcs

In [ ]:
fig = plt.figure(1)
fig.clf() # just in case one was open before
# use astropy's wcsaxes tool to create an RA/Dec image
ax = fig.add_axes([0.15, 0.1, 0.8, 0.8], projection=mywcs)
ax.set_xlabel("RA")
ax.set_ylabel("Dec")

ax.imshow(img[0].data, cmap='gray_r', interpolation='none', origin='lower',
          norm=plt.matplotlib.colors.LogNorm())



In [ ]:
data = img[0].data
hdr = img[0].header

In [ ]:
plt.figure(figsize=(14, 8))
plt.imshow(data, cmap="gray")
plt.colorbar()

plt.show()

In [ ]:
hdr